In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
import numpy as np
import random

def set_seed(val):
    torch.manual_seed(val)
    torch.cuda.manual_seed(val)
    torch.cuda.manual_seed_all(val)
    np.random.seed(val)
    random.seed(val)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

In [3]:
# Load data
merged_data = pd.read_csv('../data/merged_data.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

merged_data['contract_date'] = pd.to_datetime(merged_data['contract_year_month'].astype(str) + merged_data['contract_day'].astype(str), format='%Y%m%d')
merged_data['contract_date'] = (merged_data['contract_date'] - pd.Timestamp('2019-01-01')).dt.days

In [4]:
train_data = merged_data[merged_data['_type'] == 'train'].drop(columns=['_type'])
test_data = merged_data[merged_data['_type'] == 'test'].drop(columns=['_type', 'deposit'])

In [5]:
class TrainDataset(Dataset):
    def __init__(self, data, scaler=None):
        self.data = data
        self.X = self.data.drop(columns=['deposit']).values
        self.y = self.data['deposit'].values
        self.scaler = scaler
        
        # batch_normalization으로 대체
        # if scaler is None:
        #     self.scaler = StandardScaler()
        #     self.X = self.scaler.fit_transform(self.X)
        # else:
        #     self.scaler = scaler
        #     self.X = self.scaler.transform(self.X)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)
    
class TestDataset(Dataset):
    def __init__(self, data, scaler):
        self.data = data
        self.X = self.data.values
        
        # batch_normalization으로 대체
        # self.scaler = scaler
        # self.X = self.scaler.transform(self.X)
    
    def __len__(self):  
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32)

In [6]:
columns = ['latitude', 'longitude', 'cluster_kmeans', 'contract_date', 'deposit_mean', 'interest_rate', 'area_m2', 'floor', 'built_year', 'age', 'nearest_subway_distance_km', 'contract_type', ]
train_data = train_data[columns + ['deposit']]
test_data = test_data[columns]

In [7]:
train_data.head()

,latitude,longitude,cluster_kmeans,contract_date,deposit_mean,interest_rate,area_m2,floor,built_year,age,nearest_subway_distance_km,contract_type,deposit
0,37.054314,127.045216,0.0,175,31111.287554,1.92,84.9981,9,2019,0,0.716953,2,17000.0
1,37.054314,127.045216,0.0,450,32148.246310,1.63,84.9981,20,2019,1,0.716953,2,23000.0
2,37.054314,127.045216,0.0,452,32148.246310,1.63,84.9981,8,2019,1,0.716953,2,23000.0
3,36.964647,127.055847,1.0,195,31621.427691,1.94,59.3400,1,1986,33,3.897280,2,5000.0
4,36.972390,127.084514,1.0,101,30447.425958,2.04,59.8100,6,1995,24,2.039685,2,1800.0


In [8]:
from tqdm import tqdm
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

train_dataset = TrainDataset(train_data)
val_dataset = TrainDataset(val_data, scaler=train_dataset.scaler)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

# Define model
class Model(nn.Module):
    def __init__(self, input_size, location_feat_size, time_feat_size, location_emb_size=8, time_emb_size=4, num_heads=4):
        super().__init__()
        # 위치 정보를 embedding
        self.location_feat_size = location_feat_size
        self.time_feat_size = time_feat_size
        self.location_emb_size = location_emb_size
        self.time_emb_size = time_emb_size
        
        self.location_embedding = nn.Linear(location_feat_size, location_emb_size) 
        self.time_embedding = nn.Linear(time_feat_size, time_emb_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=(location_emb_size + time_emb_size), nhead=num_heads)
        
        # 시간 + 위치 정보를 이용한 trend
        self.fc_trend = nn.Sequential(
            nn.BatchNorm1d(location_emb_size + time_emb_size),
            nn.TransformerEncoder(self.encoder_layer, 2),
            nn.Linear(location_emb_size + time_emb_size, 1)
        )
        
        # 전체 feature를 이용한 residual
        self.fc_residual = nn.Sequential( 
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
        self.out_layer = nn.Linear(2, 1)
        
    def forward(self, x):
        # location은 1 ~ 4번째 feature
        location = x[:, :self.location_feat_size]
        location = self.location_embedding(location)
        
        time = x[:, self.location_feat_size:self.location_feat_size + self.time_feat_size]
        time = self.time_embedding(time)
        
        trend = torch.cat([location.view(-1, self.location_emb_size), time.view(-1, self.time_emb_size)], dim=1)
        trend = self.fc_trend(trend)
        
        residual = x[:, :]
        residual = self.fc_residual(residual)
        return self.out_layer(torch.cat([trend, residual], dim=1))
        
        
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Model(train_dataset.X.shape[1], 3, 3).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

cuda


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, n_epochs, patience):
    best_val_loss = float('inf')
    counter = 0
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2)
    for epoch in range(n_epochs):
        train_losses = []
        for data in tqdm(train_loader):
            optimizer.zero_grad()
            input_data, target = data
            input_data = input_data.to(device)
            target = target.to(device)
            input_data.to(device)
            output = model(input_data)
            loss = criterion(output, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        # Evaluate model
        model.eval()

        val_losses = []
        val_preds = []

        with torch.no_grad():
            for i, data in enumerate(val_loader):
                input_data, target = data
                input_data = input_data.to(device)
                target = target.to(device)
                output = model(input_data)
                # MAE Loss
                loss = criterion(output, target.view(-1, 1))
                val_losses.append(loss.item())

        val_loss = sum(val_losses) / len(val_losses)
        print(f'Epoch {epoch}, Train Loss: {sum(train_losses) / len(train_losses)}, Val Loss: {val_loss}')
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print('Early stopping')
                break
        scheduler.step()
    
    return model

model.train()
model = train_model(model, train_loader, val_loader, criterion, optimizer, device, 100, 10)

100%|██████████| 5629/5629 [01:02<00:00, 90.10it/s] 


Epoch 0, Train Loss: 8196.18968507811, Val Loss: 8494.43469689109


100%|██████████| 5629/5629 [00:58<00:00, 95.97it/s] 


Epoch 1, Train Loss: 6630.5750952968165, Val Loss: 6603.5643286271525


100%|██████████| 5629/5629 [00:59<00:00, 94.72it/s] 


...

Epoch 86, Train Loss: 4896.655882596334, Val Loss: 4918.1494451002645


100%|██████████| 5629/5629 [00:56<00:00, 98.76it/s] 


Epoch 87, Train Loss: 4896.667454315473, Val Loss: 4918.149345571344


100%|██████████| 5629/5629 [00:59<00:00, 94.78it/s] 


Epoch 88, Train Loss: 4896.674307645119, Val Loss: 4918.150370164351
Early stopping


In [12]:
torch.save(model.state_dict(), './checkpoint/custom_mlp.pth')

In [10]:
checkpoint = torch.load('./checkpoint/custom_mlp.pth')
model.load_state_dict(checkpoint)

# Test model
test_dataset = TestDataset(test_data, None)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

model.eval()
test_preds = []

with torch.no_grad():
    for i, data in enumerate(test_loader):
        input_data = data
        input_data = input_data.to(device)
        output = model(input_data)
        test_preds.append(output.cpu().numpy())
        
test_preds = np.concatenate(test_preds).flatten()

In [22]:
sample_submission['deposit'] = test_preds
sample_submission['deposit'] = sample_submission['deposit'].apply(lambda x: 1000 if x < 0 else x)
sample_submission.to_csv('mlp_submission.csv', index=False)

In [23]:
sample_submission.describe()

,index,deposit
count,150172.00000,150172.000000
mean,75085.50000,41654.463958
std,43351.06665,25865.791051
min,0.00000,695.305908
25%,37542.75000,24341.238770
50%,75085.50000,35964.191406
75%,112628.25000,52266.271484
max,150171.00000,715633.375000
